## Ames Feature Engineering and data cleansing

In [179]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
from scipy import stats
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline


#Setting for pandas to show all columns
pd.set_option('display.max_columns', None)

#### Read data and merge train and test

In [192]:
#We read the data and create a Data Frame with the whole dataset

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


# Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

# Now drop the 'Id' colum since we cannot use it as a feature to train our model.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

y_train = train['SalePrice']
X_train = train.drop('SalePrice', axis=1)
#copy of our test data in order ot recover thes dataset
X_test = test.copy()

all_data = pd.concat([X_train, X_test], ignore_index=True)
#all_data.info()


In [181]:
#Inspect which columns have null values

all_data.isnull().sum().sort_values(ascending=False)

PoolQC           2909
MiscFeature      2814
Alley            2721
Fence            2348
FireplaceQu      1420
LotFrontage       486
GarageCond        159
GarageQual        159
GarageYrBlt       159
GarageFinish      159
GarageType        157
BsmtCond           82
BsmtExposure       82
BsmtQual           81
BsmtFinType2       80
BsmtFinType1       79
MasVnrType         24
MasVnrArea         23
MSZoning            4
BsmtHalfBath        2
Utilities           2
Functional          2
BsmtFullBath        2
BsmtFinSF2          1
BsmtFinSF1          1
Exterior2nd         1
BsmtUnfSF           1
TotalBsmtSF         1
Exterior1st         1
SaleType            1
                 ... 
YearRemodAdd        0
YearBuilt           0
SaleCondition       0
HeatingQC           0
ExterQual           0
ExterCond           0
YrSold              0
MoSold              0
MiscVal             0
PoolArea            0
ScreenPorch         0
3SsnPorch           0
EnclosedPorch       0
OpenPorchSF         0
WoodDeckSF

#### Adressing null values

We impute 'Unavaiblable' to all text columns with null values and 0 to all the numeric ones.

In [182]:
#Impute value 0 to the numeric columns and unavailable to the object ones

for c in all_data.columns:
    if all_data[c].dtype == 'object':
        all_data[c] = all_data[c].fillna('Unavailable')
    else:
        all_data[c] = all_data[c].fillna(0)       
            

#all_data.isnull().sum().sort_values(ascending=False)

all_data.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,30,C (all),60.0,7879,Pave,Unavailable,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,4,5,1920,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,495.0,Unf,0.0,225.0,720.0,GasA,TA,N,FuseA,720,0,0,720,0.0,0.0,1,0,2,1,TA,4,Typ,0,Unavailable,Unavailable,0.0,Unavailable,0.0,0.0,Unavailable,Unavailable,N,0,523,115,0,0,0,Unavailable,GdWo,Unavailable,0,11,2009,WD,Abnorml
1,20,C (all),50.0,9000,Pave,Unavailable,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,2,3,1949,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,TA,CBlock,TA,TA,Av,BLQ,50.0,Unf,0.0,430.0,480.0,GasA,TA,N,FuseA,480,0,0,480,1.0,0.0,0,0,1,1,TA,4,Typ,0,Unavailable,Detchd,1958.0,Unf,1.0,308.0,TA,TA,Y,0,0,0,0,0,0,Unavailable,Unavailable,Unavailable,0,10,2006,WD,Abnorml
2,50,RM,50.0,5925,Pave,Unavailable,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,3,6,1910,1950,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,600.0,600.0,Grav,Fa,N,SBrkr,600,368,0,968,0.0,0.0,1,0,2,1,TA,6,Typ,0,Unavailable,Unavailable,0.0,Unavailable,0.0,0.0,Unavailable,Unavailable,Y,0,0,0,0,0,0,Unavailable,GdWo,Unavailable,0,5,2009,WD,Abnorml
3,20,RL,50.0,5000,Pave,Unavailable,Reg,Low,AllPub,Inside,Mod,BrkSide,Norm,Norm,1Fam,1Story,1,3,1946,1950,Gable,CompShg,VinylSd,VinylSd,None,0.0,Fa,Fa,Slab,Unavailable,Unavailable,Unavailable,Unavailable,0.0,Unavailable,0.0,0.0,0.0,GasA,Fa,N,FuseF,334,0,0,334,0.0,0.0,1,0,1,1,Fa,2,Typ,0,Unavailable,Unavailable,0.0,Unavailable,0.0,0.0,Unavailable,Unavailable,N,0,0,0,0,0,0,Unavailable,Unavailable,Unavailable,0,1,2007,WD,Normal
4,70,C (all),50.0,8500,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Feedr,Norm,1Fam,2Story,4,4,1920,1950,Gambrel,CompShg,BrkFace,BrkFace,None,0.0,TA,Fa,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,649.0,649.0,GasA,TA,N,SBrkr,649,668,0,1317,0.0,0.0,1,0,3,1,TA,6,Typ,0,Unavailable,Detchd,1920.0,Unf,1.0,250.0,TA,Fa,N,0,54,172,0,0,0,Unavailable,MnPrv,Unavailable,0,7,2008,WD,Normal


#### Facrtorizing

We categorize the columns that although being numeric represent categories. This way we avoid classification methods like trees to give strange results, i.e: an Overall Quality of 4.5 means actually not much in this context. 

We evaluated which columns needed to be changed and transformed to categories.

In [193]:
#Creating categoriesof  numeric columns that are actually categories

#MSSubClass
#OverallQual
#OverallCond
#GarageCars
#YrSold

all_data["MSSubClass"]= all_data["MSSubClass"].astype('category')
all_data["OverallQual"]= all_data["OverallQual"].astype('category')
all_data["OverallCond"]= all_data["OverallCond"].astype('category')
all_data["GarageCars"]= all_data["GarageCars"].astype('category')
all_data["YrSold"]= all_data["YrSold"].astype('category')

#Chekcing trnasformation has been done correctly
all_data.dtypes.head(10)

MSSubClass     category
MSZoning         object
LotFrontage     float64
LotArea           int64
Street           object
Alley            object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
dtype: object

### Feature engineering

After assessing some of the variables we believe that some of them can be combined and this way affect in a greater proportion to the Sale Price than not as separated ones. Therefore we combine this.

Also there are cases that what affects more the price is the fact that there is a binary classification, tehre is Pool yes or no for example than not the actual characteristics fo the pool or Fireplace. 

In [184]:
#Lets feature engineeer two variables from numericla ones to Yes/No
#We are interested in finding out if we have pool or not, if we have fireplace or not

all_data['Fireplaces'] = np.where(all_data['Fireplaces'] == 0, 'No', 'Yes')

all_data.Fireplaces.value_counts()

Yes    1499
No     1420
Name: Fireplaces, dtype: int64

In [185]:
#Method 2 to achieve the same purpose this time with the loc function.

all_data['PoolArea'] = all_data['PoolArea'].astype(str)

all_data.loc[all_data.PoolArea!='0','PoolArea'] = 'Yes'
all_data.loc[all_data.PoolArea=='0','PoolArea'] = 'No'

all_data.PoolArea.value_counts()


No     2906
Yes      13
Name: PoolArea, dtype: int64

#### Combine some of the data columns

In [194]:
#We combine all the total sqf in order to have jsut one variable

all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
#and drop the columns
all_data.drop(['TotalBsmtSF','1stFlrSF','2ndFlrSF'], inplace=True, axis=1)


In [171]:
#Then we do the same for the baths
all_data['TotalBath'] = all_data['BsmtFullBath'] + all_data['FullBath'] + all_data['HalfBath'] + all_data['BsmtHalfBath']
#and drop the columns
all_data.drop(['HalfBath','FullBath','BsmtHalfBath','BsmtFullBath'], inplace=True, axis=1)

In [187]:
#lets aggreagete all teh porchs sf into one

all_data['TotalPorchSF']  = all_data['WoodDeckSF'] + all_data['OpenPorchSF'] + all_data['EnclosedPorch']  + all_data['3SsnPorch'] + all_data['ScreenPorch']
all_data.drop(['WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'], inplace=True, axis=1)

### Categorical variables

We need to dummify categorical variables so we can use them in linear models. This can be done dummifying them, or using techniques as one hot eoncoder, or label count econding. 

We first run a quick get_dummies to see the amount of columns we get and will take it from there.

In [189]:
one_hot_df = pd.get_dummies(all_data, drop_first=True, dummy_na=True)
one_hot_df.head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageArea,MiscVal,MoSold,TotalSF,TotalPorchSF,MSSubClass_30.0,MSSubClass_40.0,MSSubClass_45.0,MSSubClass_50.0,MSSubClass_60.0,MSSubClass_70.0,MSSubClass_75.0,MSSubClass_80.0,MSSubClass_85.0,MSSubClass_90.0,MSSubClass_120.0,MSSubClass_150.0,MSSubClass_160.0,MSSubClass_180.0,MSSubClass_190.0,MSSubClass_nan,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MSZoning_Unavailable,MSZoning_nan,Street_Pave,Street_nan,Alley_Pave,Alley_Unavailable,Alley_nan,LotShape_IR2,LotShape_IR3,LotShape_Reg,LotShape_nan,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandContour_nan,Utilities_NoSeWa,Utilities_Unavailable,Utilities_nan,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LotConfig_nan,LandSlope_Mod,LandSlope_Sev,LandSlope_nan,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Neighborhood_nan,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition1_nan,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Condition2_nan,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BldgType_nan,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,HouseStyle_nan,OverallQual_2.0,OverallQual_3.0,OverallQual_4.0,OverallQual_5.0,OverallQual_6.0,OverallQual_7.0,OverallQual_8.0,OverallQual_9.0,OverallQual_10.0,OverallQual_nan,OverallCond_2.0,OverallCond_3.0,OverallCond_4.0,OverallCond_5.0,OverallCond_6.0,OverallCond_7.0,OverallCond_8.0,OverallCond_9.0,OverallCond_nan,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofStyle_nan,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofMatl_nan,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_Unavailable,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior1st_nan,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_Unavailable,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Exterior2nd_nan,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MasVnrType_Unavailable,MasVnrType_nan,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterQual_nan,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,ExterCond_nan,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Foundation_nan,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtQual_Unavailable,BsmtQual_nan,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtCond_Unavailable,BsmtCond_nan,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_Unavailable,BsmtExposure_nan,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unavailable,BsmtFinType1_Unf,BsmtFinType1_nan,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unavailable,BsmtFinType2_Unf,BsmtFinType2_nan,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating_nan,HeatingQC_F

In [195]:
#Quick PCA to see how many we get after the get_dummies!

#Standardize the data.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(one_hot_df)

# Apply transform to both the training set and the test set.
train_PCA = scaler.transform(one_hot_df)

from  sklearn.decomposition import PCA

#Calculate the PCA so the selected variables explain at least 90% of the variance of our model 0.90
pca = PCA(0.90)

#Fit PCA on training set.
pca.fit(train_PCA)

#
print(pca.n_components_)
print(pca.components_)


#Apply the mapping (transform) to both the training set and the test set.
train_PCA = pca.transform(train_PCA)

#which is the variance ratio of each component?
print(pca.explained_variance_ratio_)

#and the total cumulutative
cumsum = np.cumsum(pca.explained_variance_ratio_*100)
print(cumsum) # we see reaches 90%

166
[[ 2.57294489e-02  3.39535363e-02  1.98671042e-01 ... -3.86237512e-02
   9.51393621e-02 -0.00000000e+00]
 [ 2.99380237e-02 -3.47088028e-02  3.07364194e-03 ... -8.53176280e-02
   9.11323346e-02 -0.00000000e+00]
 [ 3.50470498e-02  9.99317125e-02  5.33906074e-02 ... -1.86565850e-03
  -2.10892816e-02  0.00000000e+00]
 ...
 [ 6.29268783e-02  4.93823476e-02 -2.06778897e-03 ... -1.14799078e-02
  -1.16022406e-02  0.00000000e+00]
 [ 2.36303694e-01 -4.84969306e-02 -2.11306448e-02 ...  5.64115911e-04
   1.33241067e-02 -0.00000000e+00]
 [-6.70040768e-02 -1.56278138e-04 -1.04478565e-02 ...  5.17902952e-03
   2.23213756e-04  0.00000000e+00]]
[0.05983272 0.02861076 0.02447794 0.020101   0.01923162 0.01729383
 0.01395872 0.01364463 0.01114792 0.01068977 0.0104976  0.01028856
 0.01006477 0.00943709 0.00905235 0.00878692 0.0085053  0.00841334
 0.00833222 0.00815221 0.00796212 0.0078591  0.00769478 0.00756669
 0.00748811 0.00734289 0.00718464 0.0070733  0.00703516 0.00686483
 0.00667644 0.00657128 0.